In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure

# Base directory where the main folders are located
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Severities"

# Main folders with their subfolders
main_folders = {
    "Cataract": ["Cataract_low", "Cataract_medium", "Cataract_high"],
    "Diabetic Retinopathy": ["Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high"],
    "Normal": [],  # No subfolders for Normal
    "Glaucoma": ["Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"],
}

# Dictionary to store features for all subfolders
features_by_folder = {}

# Function to extract HOG features from a grayscale image
def extract_hog_features(image):
    # Extract HOG features and a visualization image
    hog_features, hog_image = hog(image, visualize=True, pixels_per_cell=(16, 16), cells_per_block=(1, 1), feature_vector=True)
    
    # Rescale the HOG image for visualization (optional)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    return hog_features, hog_image_rescaled

# Function to read images from a folder and extract HOG features
def read_and_extract_features(folder_path):
    # List all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Store HOG features from this folder
    hog_features_list = []

    # Loop through each image and extract HOG features
    for image_file in image_files:
        img_path = os.path.join(folder_path, image_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            # Extract HOG features
            hog_features, _ = extract_hog_features(image)

            # Append the HOG features to the list
            hog_features_list.append(hog_features)
    
    return hog_features_list

# Loop through each main folder and its subfolders to extract HOG features
for main_folder, subfolder_list in main_folders.items():
    if subfolder_list:
        # Process each subfolder under the main folder
        for subfolder in subfolder_list:
            print(f"Processing subfolder: {main_folder}/{subfolder}")
            folder_path = os.path.join(base_dir, main_folder, subfolder)

            # Extract features from the current subfolder
            features = read_and_extract_features(folder_path)

            # Store the extracted features in the dictionary
            features_by_folder[f"{main_folder}/{subfolder}"] = features

            # Provide feedback on the number of images processed
            print(f"Extracted HOG features from {len(features)} images in '{main_folder}/{subfolder}'")
    else:
        # If no subfolders, process the main folder directly (like Normal)
        print(f"Processing main folder: {main_folder}")
        folder_path = os.path.join(base_dir, main_folder)

        # Extract features from the current main folder
        features = read_and_extract_features(folder_path)

        # Store the extracted features in the dictionary
        features_by_folder[main_folder] = features

        # Provide feedback on the number of images processed
        print(f"Extracted HOG features from {len(features)} images in '{main_folder}'")

# Now 'features_by_folder' contains HOG features from all specified folders and subfolders.


Processing subfolder: Cataract/Cataract_low
Extracted HOG features from 382 images in 'Cataract/Cataract_low'
Processing subfolder: Cataract/Cataract_medium
Extracted HOG features from 350 images in 'Cataract/Cataract_medium'
Processing subfolder: Cataract/Cataract_high
Extracted HOG features from 300 images in 'Cataract/Cataract_high'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_low
Extracted HOG features from 317 images in 'Diabetic Retinopathy/Diabetic Retinopathy_low'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_medium
Extracted HOG features from 401 images in 'Diabetic Retinopathy/Diabetic Retinopathy_medium'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_high
Extracted HOG features from 380 images in 'Diabetic Retinopathy/Diabetic Retinopathy_high'
Processing main folder: Normal
Extracted HOG features from 1074 images in 'Normal'
Processing subfolder: Glaucoma/Glaucoma_low
Extracted HOG features from 354 images in 'Glaucoma/G

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

# Base directory for the dataset
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Severities"

# Main folders with their subfolders
main_folders = {
    "Cataract": ["Cataract_low", "Cataract_medium", "Cataract_high"],
    "Diabetic Retinopathy": ["Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high"],
    "Normal": [],  # No subfolders for Normal
    "Glaucoma": ["Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"],
}

# Prepare the data generator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Example augmentation settings
    horizontal_flip=True  # Example augmentation setting
)

# Training dataset from specified subfolders
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # EfficientNetB0 input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation dataset from specified subfolders
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # EfficientNetB0 input size
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the pre-trained EfficientNetB0 model without the top layer (for transfer learning)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for our specific problem
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Flattening
x = layers.Dense(256, activation='relu')(x)  # Fully connected layer
x = layers.Dropout(0.5)(x)  # Dropout for regularization
output = layers.Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for categorical classification

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base EfficientNet layers
for layer in base_model.layers:
    layer.trainable = False  # Use pre-trained weights

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {accuracy:.2f}")


Found 3371 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
16705208/16705208 [==============================] - 9s 1us/step
Epoch 1/10
106/106 [==============================] - 1294s 12s/step - loss: 1.4194 - accuracy: 0.2477 - val_loss: 1.3859 - val_accuracy: 0.2607
Epoch 2/10
106/106 [==============================] - 1274s 12s/step - loss: 1.3865 - accuracy: 0.2480 - val_loss: 1.3860 - val_accuracy: 0.2548
Epoch 3/10
106/106 [==============================] - 1279s 12s/step - loss: 1.3861 - accuracy: 0.2530 - val_loss: 1.3859 - val_accuracy: 0.2607
Epoch 4/10
106/106 [==============================] - 1309s 12s/step - loss: 1.3859 - accuracy: 0.2608 - val_loss: 1.3858 - val_accuracy: 0.2607
Epoch 5/10
106/106 [==============================] - 1279s 12s/step - loss: 1.3863 - accuracy: 0.2608 - val_loss: 1.3858 - val_accuracy: 0.2607
Epoch 6/10
106/106 [==============================] - 1330s 12s/step - loss: 1.3859 - accuracy: 0.2548 - val_loss: 1.3858 - val

In [4]:
import numpy as np